In [1]:
import os
from dotenv import load_dotenv
from typing import List, Dict, Tuple, Any, TypedDict


In [2]:
apikey = os.getenv("HEVY_API_KEY")

In [3]:
apikey

'1052b925-d1d3-47ef-9049-fd56c3d2e367'

In [7]:
last_volume = 1394
volume_increase = 0.05

In [8]:
def calculate_next_volume_goal(last_volume: int, increment_percentage: float) -> int:
    new_volume = int(last_volume + (last_volume * increment_percentage))

    return new_volume

In [9]:
calculate_next_volume_goal(last_volume, volume_increase)

1463

In [10]:
url = "https://api.hevyapp.com/v1/workouts?page=1"
headers = {
    "accept": "application/json",
    "api-key": apikey
}
response = requests.get(url=url, headers=headers).json()

In [11]:
response

{'page': 1,
 'page_count': 9,
 'workouts': [{'id': '44bd15b8-a5d3-4893-81ff-5c0f0c73614f',
   'title': 'CBSA',
   'description': '',
   'start_time': '2025-07-18T18:09:24+00:00',
   'end_time': '2025-07-18T19:09:40+00:00',
   'updated_at': '2025-07-18T19:09:42.436Z',
   'created_at': '2025-07-18T19:09:42.436Z',
   'exercises': [{'index': 0,
     'title': 'Chest Press (Machine)',
     'notes': '',
     'exercise_template_id': '7EB3F7C3',
     'superset_id': None,
     'sets': [{'index': 0,
       'type': 'warmup',
       'weight_kg': 30,
       'reps': 10,
       'distance_meters': None,
       'duration_seconds': None,
       'rpe': None,
       'custom_metric': None},
      {'index': 1,
       'type': 'normal',
       'weight_kg': 40,
       'reps': 10,
       'distance_meters': None,
       'duration_seconds': None,
       'rpe': None,
       'custom_metric': None},
      {'index': 2,
       'type': 'normal',
       'weight_kg': 40,
       'reps': 10,
       'distance_meters': None,


In [15]:
workouts = response['workouts']

In [23]:
workouts[0]['exercises']

[{'index': 0,
  'title': 'Chest Press (Machine)',
  'notes': '',
  'exercise_template_id': '7EB3F7C3',
  'superset_id': None,
  'sets': [{'index': 0,
    'type': 'warmup',
    'weight_kg': 30,
    'reps': 10,
    'distance_meters': None,
    'duration_seconds': None,
    'rpe': None,
    'custom_metric': None},
   {'index': 1,
    'type': 'normal',
    'weight_kg': 40,
    'reps': 10,
    'distance_meters': None,
    'duration_seconds': None,
    'rpe': None,
    'custom_metric': None},
   {'index': 2,
    'type': 'normal',
    'weight_kg': 40,
    'reps': 10,
    'distance_meters': None,
    'duration_seconds': None,
    'rpe': None,
    'custom_metric': None},
   {'index': 3,
    'type': 'normal',
    'weight_kg': 45,
    'reps': 10,
    'distance_meters': None,
    'duration_seconds': None,
    'rpe': None,
    'custom_metric': None},
   {'index': 4,
    'type': 'normal',
    'weight_kg': 55,
    'reps': 8,
    'distance_meters': None,
    'duration_seconds': None,
    'rpe': None,


In [56]:
all_exercises = set([ex['title'] for ex in [w for w in response['workouts']['exercises']]])

TypeError: list indices must be integers or slices, not str

In [52]:
all_exercises

{'Afternoon workout 💪',
 'BCSA',
 'CBSA',
 'Corila',
 'Evening workout 🏋️',
 'LGA',
 'Leg',
 'Limbεροπουλος'}

In [71]:
[[ex['title'] for ex in w['exercises']] for w in response['workouts']]

[['Chest Press (Machine)',
  'Seated Cable Row - Bar Wide Grip',
  'Overhead Press (Smith Machine)',
  'Hammer Curl (Cable)',
  'Triceps Pushdown'],
 ['Leg Press Horizontal (Machine)',
  'Leg Extension (Machine)',
  'Lying Leg Curl (Machine)',
  'Bicep Curl (Cable)',
  'Triceps Extension (Cable)',
  'Dead Hang'],
 ['Glute Bridge', 'Dead Bug', 'Sit Up', 'Superman'],
 ['Lat Pulldown (Cable)',
  'Chest Press (Machine)',
  'Lateral Raise (Dumbbell)',
  'Bicep Curl (Cable)',
  'Triceps Pushdown'],
 ['Leg Press Horizontal (Machine)',
  'Leg Extension (Machine)',
  'Lying Leg Curl (Machine)',
  'Bicep Curl (Cable)',
  'Triceps Extension (Cable)',
  'Dead Hang'],
 ['Incline Bench Press (Dumbbell)',
  'Lat Pulldown (Cable)',
  'Lateral Raise (Dumbbell)',
  'Bicep Curl (Cable)',
  'Triceps Pushdown',
  'Leg Press Horizontal (Machine)'],
 ['Incline Bench Press (Dumbbell)',
  'Chest Press (Machine)',
  'Pull Up',
  'Lat Pulldown (Machine)',
  'Lateral Raise (Dumbbell)'],
 ['Bicep Curl (Cable)',
  

In [64]:
[w['title'] for w in response['workouts']]

['CBSA',
 'LGA',
 'Corila',
 'BCSA',
 'Leg',
 'Afternoon workout 💪',
 'Evening workout 🏋️',
 'Limbεροπουλος',
 'Afternoon workout 💪',
 'Afternoon workout 💪']

In [117]:
import math
import requests
from typing import Any, Dict, List, Optional, TypedDict, Tuple

class ExerciseData(TypedDict):
    exercise: str
    sets: List[Tuple[int, float]]
    volume: int


class Hevy:
    def __init__(self, apikey):
        self.apikey = apikey
        self.all_workouts = {}
        self.all_exercises = {}

    # ----------------------------------
    # Fetching functions
    # ----------------------------------
    def fetch_first_page_of_data(self):
        url = "https://api.hevyapp.com/v1/workouts?page=1"
        headers = {
            "accept": "application/json",
            "api-key": self.apikey
        }
        response = requests.get(url=url, headers=headers)

        if response.status_code == 401:
            raise ValueError("Invalid API key – Hevy returned 401 Unauthorized")
        if not response.ok:
            raise RuntimeError(f"Hevy API error {response.status_code}: {response.text[:120]}")
        try:
            payload = response.json()
        except ValueError as e:
            raise RuntimeError("Hevy response was not JSON, check the key/network") from e

        return payload


    def get_all_workouts(self):
        data = self.fetch_first_page_of_data()

        all_workouts = set([ex['title'] for ex in [w for w in data['workouts']]])
        self.all_workouts = all_workouts
        return all_workouts
    

    def get_all_exercises(self):
        data = self.fetch_first_page_of_data()

        all_exercises = [[ex['title'] for ex in w['exercises']] for w in data['workouts']]
        unique_exercises = sum(all_exercises, [])

        self.all_exercises = unique_exercises
        return all_exercises
    
    
    def fetch_last_workout(self, workout_title: Optional[str] = None) -> dict:
        """
        Fetch the last workout from Hevy API.
        If workout_title is provided, returns the most recent workout with that title.
        Otherwise, returns the most recent workout.

        Parameters:
        -----------
        apikey : str
            Your Hevy API key.
        workout_title : Optional[str]
            Title of the workout to filter for.

        Returns:
        --------
        dict
            The workout data.
        """

        payload = self.fetch_first_page_of_data()

        if workout_title:
            filtered_workouts = [w for w in payload['workouts'] if w['title'] == workout_title]
            last_workout_data = filtered_workouts[0] if filtered_workouts else None
        else:
            last_workout_data = payload['workouts'][0] if payload['workouts'] else None

        return last_workout_data
    

    def get_exercise_last_data(self, exercise_name: str) -> dict:
        payload = self.fetch_first_page_of_data()

        if exercise_name not in self.all_exercises:
            return {'error':"Exercise not found in data"}
        else:
            for workout in payload['workouts']:
                for exercise in workout['exercises']:
                    if exercise['title'] == exercise_name:
                        return exercise


    def calculate_exercise_volume(self, exercise_data: List[dict]) -> int:
        total_volume = 0
        for s in exercise_data:
            total_volume += s['weight_kg'] * s['reps'] if s['weight_kg'] and s['reps'] else 0
        
        return total_volume
    

    def structure_workout_data(self, workout_data: dict) -> List[ExerciseData]:
        schema = []
        for exercice in workout_data['exercises']:

            exercise_volume = self.calculate_exercise_volume(exercice['sets'])

            exercise_data = {
                "exercise": exercice['title'],
                "sets": [(s['reps'], s['weight_kg']) for s in exercice['sets']],
                "volume": exercise_volume
            }
            schema.append(exercise_data)
        return schema



    def optimize_weight_and_reps(
        self,
        base_sets: List[Tuple[int,float]],
        delta: float,
        avg_w: float,
        max_pct_wb: float,
        rep_floor: int,
        rep_cap:   int,
        exercise_name: str
    ) -> Optional[Dict[str,Any]]:
        """
        Try all 1.25kg bumps up to max_pct_wb * avg_w (or only 9kg for Leg Press),
        plus greedy-rep, to hit or slightly overshoot delta.
        """
        n = len(base_sets)
        # pick weight bump candidates
        if exercise_name == 'Leg Press Horizontal (Machine)':
            candidates = [9.0]
        else:
            max_w_inc = math.floor((avg_w*max_pct_wb)/1.25)*1.25
            candidates = [1.25 * i for i in range(1, int(max_w_inc/1.25)+1)]

        best = None
        for w_inc in candidates:
            # apply uniform weight bump
            bumped = [(r, w + w_inc) for r,w in base_sets]
            added_wvol = sum(r * w_inc for r,_ in base_sets)
            rem1 = delta - added_wvol
            if rem1 <= 0:
                return {
                    'w_inc': w_inc,
                    'bump_reps': [0]*n,
                    'total_added': added_wvol,
                    'overshoot': added_wvol - delta,
                    'final_sets': bumped
                }

            # greedy‐rep on bumped
            bump_r = [0]*n
            cap_extra = [rep_cap - r for r,_ in base_sets]
            # enforce rep_floor
            repfloor_added = 0.0
            for i,(r,w) in enumerate(bumped):
                if r < rep_floor:
                    need = min(rep_floor - r, cap_extra[i])
                    bump_r[i] = need
                    cap_extra[i] -= need
                    repfloor_added += need * w
            rem2 = rem1 - repfloor_added

            # one-rep at a time on heaviest
            added_rvol = repfloor_added
            order = sorted(range(n), key=lambda i: bumped[i][1], reverse=True)
            while rem2 > 0:
                for i in order:
                    if cap_extra[i] > 0:
                        bump_r[i] += 1
                        cap_extra[i] -= 1
                        added_rvol += bumped[i][1]
                        rem2 -= bumped[i][1]
                        break
                else:
                    break

            total_added = added_wvol + added_rvol
            overshoot = total_added - delta

            if any(bump_r) and overshoot >= -1e-6:
                final = [(r + bump_r[i], w + w_inc) for i,(r,w) in enumerate(base_sets)]
                cand = {
                    'w_inc': w_inc,
                    'bump_reps': bump_r,
                    'total_added': total_added,
                    'overshoot': overshoot,
                    'final_sets': final
                }
                if (best is None or
                    cand['overshoot'] < best['overshoot'] - 1e-6 or
                (abs(cand['overshoot']-best['overshoot'])<1e-6 and w_inc < best['w_inc'])):
                    best = cand

        return best

    def get_optimized_options(
        self,
        data: Dict[str,Any],
        volume_perc: float,
        max_pct_weight_bump: float = 0.10,
        rep_floor: int = 6,
        rep_cap:   int = 12
    ) -> Dict[str,Any]:
        orig = [(r,w) for r,w in data['sets'] if r is not None and w is not None]
        # handle exercises with no valid weight data
        if not orig:
            return {
                'delta':          0.0,
                'target_volume':  0.0,
                'phases':         [],
                'final_sets':     []
            }

        total_vol    = sum(r*w for r,w in orig)
        target_vol   = total_vol * (1 + volume_perc)
        delta        = target_vol - total_vol
        n            = len(orig)
        avg_w        = sum(w for _,w in orig) / n
        ex_name      = data.get('exercise','')
        result       = {'delta': delta, 'target_volume': target_vol, 'phases': []}

        # Phase 0: flatten >rep_cap
        lost = 0.0
        base_sets = []
        for r,w in orig:
            if r > rep_cap:
                lost += (r - rep_cap)*w
                base_sets.append((rep_cap,w))
            else:
                base_sets.append((r,w))
        vol0 = sum(r*w for r,w in base_sets)
        rem0 = target_vol - vol0
        result['phases'].append({
            'phase': 0,
            'lost_volume': lost,
            'base_sets': base_sets,
            'remaining': rem0
        })
        if rem0 <= 1e-6:
            result['final_sets'] = base_sets
            return result

        # Phase 1+2: optimized weight+reps
        best = self.optimize_weight_and_reps(
            base_sets, rem0, avg_w, max_pct_weight_bump,
            rep_floor, rep_cap, ex_name
        )
        if best:
            rem1 = rem0 - best['total_added']
            result['phases'].append({
                'phase': 1,
                'w_inc': best['w_inc'],
                'bump_reps': best['bump_reps'],
                'added_volume': best['total_added'],
                'overshoot': best['overshoot'],
                'final_sets': best['final_sets'],
                'remaining': rem1
            })
            if rem1 <= 1e-6:
                result['final_sets'] = best['final_sets']
                return result
            sets_so_far = best['final_sets']
        else:
            rem1 = rem0
            sets_so_far = base_sets

        # Phase 3: add new sets at rep_cap
        unit = rep_cap * avg_w
        full = int(rem1 // unit)
        rem3 = rem1 - full*unit
        plan = [(rep_cap, round(avg_w,2)) for _ in range(full)]
        if rem3 > 1e-6:
            rp = min(rep_cap, math.ceil(rem3/avg_w))
            plan.append((rp, round(avg_w,2)))
        added3 = sum(r*w for r,w in plan)
        final = sets_so_far + plan

        result['phases'].append({
            'phase': 2,
            'full_new_sets': full,
            'partial_new_set': plan[full:] or None,
            'added_volume': added3,
            'new_sets': final,
            'remaining': rem1 - added3
        })
        result['final_sets'] = final
        return result


In [118]:
client = Hevy(apikey=apikey)

In [119]:
client.get_all_exercises()

ConnectionError: HTTPSConnectionPool(host='api.hevyapp.com', port=443): Max retries exceeded with url: /v1/workouts?page=1 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7c442c131bd0>: Failed to resolve 'api.hevyapp.com' ([Errno -3] Temporary failure in name resolution)"))

In [37]:
def get_exercise_last_data(apikey: str, exercise: str) -> dict:
    url = "https://api.hevyapp.com/v1/workouts?page=1"
    headers = {
        "accept": "application/json",
        "api-key": apikey
    }
    response = requests.get(url=url, headers=headers).json()

    all_exercises = set([ex['title'] for ex in [w for w in response['workouts']]])

    if exercise not in all_exercises:
        return {'error':"Exercise not found in data"}
    else:
        for workout in response['workouts']:
            for exercise in workout:
                if exercise['title'] == exercise:
                    return exercise
        




In [ ]:
get_exercise_last_data(apikey=apikey, exercise="")

{'error': 'Exercise not found in data'}

In [5]:
from typing import Optional
import requests

def fetch_last_workout(apikey: str, workout_title: Optional[str] = None) -> dict:
    """
    Fetch the last workout from Hevy API.
    If workout_title is provided, returns the most recent workout with that title.
    Otherwise, returns the most recent workout.

    Parameters:
    -----------
    apikey : str
        Your Hevy API key.
    workout_title : Optional[str]
        Title of the workout to filter for.

    Returns:
    --------
    dict
        The workout data.
    """
    url = "https://api.hevyapp.com/v1/workouts?page=1"
    headers = {
        "accept": "application/json",
        "api-key": apikey
    }
    response = requests.get(url=url, headers=headers).json()

    if workout_title:
        filtered_workouts = [w for w in response['workouts'] if w['title'] == workout_title]
        last_workout_data = filtered_workouts[0] if filtered_workouts else None
    else:
        last_workout_data = response['workouts'][0] if response['workouts'] else None

    return last_workout_data


In [8]:
workout = fetch_last_workout(apikey=apikey)

In [9]:
workout

{'id': '44bd15b8-a5d3-4893-81ff-5c0f0c73614f',
 'title': 'CBSA',
 'description': '',
 'start_time': '2025-07-18T18:09:24+00:00',
 'end_time': '2025-07-18T19:09:40+00:00',
 'updated_at': '2025-07-18T19:09:42.436Z',
 'created_at': '2025-07-18T19:09:42.436Z',
 'exercises': [{'index': 0,
   'title': 'Chest Press (Machine)',
   'notes': '',
   'exercise_template_id': '7EB3F7C3',
   'superset_id': None,
   'sets': [{'index': 0,
     'type': 'warmup',
     'weight_kg': 30,
     'reps': 10,
     'distance_meters': None,
     'duration_seconds': None,
     'rpe': None,
     'custom_metric': None},
    {'index': 1,
     'type': 'normal',
     'weight_kg': 40,
     'reps': 10,
     'distance_meters': None,
     'duration_seconds': None,
     'rpe': None,
     'custom_metric': None},
    {'index': 2,
     'type': 'normal',
     'weight_kg': 40,
     'reps': 10,
     'distance_meters': None,
     'duration_seconds': None,
     'rpe': None,
     'custom_metric': None},
    {'index': 3,
     'type': 

In [11]:
last_workout = fetch_last_workout(apikey)

In [12]:
last_workout

{'id': 'd586336c-0371-42c4-8124-c6872b4f5c83',
 'title': 'Corila',
 'description': '',
 'start_time': '2025-07-16T18:42:27+00:00',
 'end_time': '2025-07-16T18:58:58+00:00',
 'updated_at': '2025-07-16T18:59:01.112Z',
 'created_at': '2025-07-16T18:59:01.112Z',
 'exercises': [{'index': 0,
   'title': 'Glute Bridge',
   'notes': '',
   'exercise_template_id': 'CDA23948',
   'superset_id': None,
   'sets': [{'index': 0,
     'type': 'normal',
     'weight_kg': None,
     'reps': 15,
     'distance_meters': None,
     'duration_seconds': None,
     'rpe': None,
     'custom_metric': None},
    {'index': 1,
     'type': 'normal',
     'weight_kg': None,
     'reps': 15,
     'distance_meters': None,
     'duration_seconds': None,
     'rpe': None,
     'custom_metric': None},
    {'index': 2,
     'type': 'normal',
     'weight_kg': None,
     'reps': 15,
     'distance_meters': None,
     'duration_seconds': None,
     'rpe': None,
     'custom_metric': None}]},
  {'index': 1,
   'title': 'D

In [17]:
last_workout['exercises'][0]

{'index': 0,
 'title': 'Leg Press Horizontal (Machine)',
 'notes': '',
 'exercise_template_id': '0EB695C9',
 'superset_id': None,
 'sets': [{'index': 0,
   'type': 'warmup',
   'weight_kg': 55,
   'reps': 14,
   'distance_meters': None,
   'duration_seconds': None,
   'rpe': None,
   'custom_metric': None},
  {'index': 1,
   'type': 'normal',
   'weight_kg': 64,
   'reps': 15,
   'distance_meters': None,
   'duration_seconds': None,
   'rpe': None,
   'custom_metric': None},
  {'index': 2,
   'type': 'normal',
   'weight_kg': 82,
   'reps': 17,
   'distance_meters': None,
   'duration_seconds': None,
   'rpe': None,
   'custom_metric': None},
  {'index': 3,
   'type': 'normal',
   'weight_kg': 91,
   'reps': 14,
   'distance_meters': None,
   'duration_seconds': None,
   'rpe': None,
   'custom_metric': None},
  {'index': 4,
   'type': 'normal',
   'weight_kg': 100,
   'reps': 12,
   'distance_meters': None,
   'duration_seconds': None,
   'rpe': None,
   'custom_metric': None},
  {'in

In [ ]:
last_workout['exercises'][0]['sets']

[{'index': 0,
  'type': 'warmup',
  'weight_kg': 55,
  'reps': 14,
  'distance_meters': None,
  'duration_seconds': None,
  'rpe': None,
  'custom_metric': None},
 {'index': 1,
  'type': 'normal',
  'weight_kg': 64,
  'reps': 15,
  'distance_meters': None,
  'duration_seconds': None,
  'rpe': None,
  'custom_metric': None},
 {'index': 2,
  'type': 'normal',
  'weight_kg': 82,
  'reps': 17,
  'distance_meters': None,
  'duration_seconds': None,
  'rpe': None,
  'custom_metric': None},
 {'index': 3,
  'type': 'normal',
  'weight_kg': 91,
  'reps': 14,
  'distance_meters': None,
  'duration_seconds': None,
  'rpe': None,
  'custom_metric': None},
 {'index': 4,
  'type': 'normal',
  'weight_kg': 100,
  'reps': 12,
  'distance_meters': None,
  'duration_seconds': None,
  'rpe': None,
  'custom_metric': None},
 {'index': 5,
  'type': 'normal',
  'weight_kg': 100,
  'reps': 12,
  'distance_meters': None,
  'duration_seconds': None,
  'rpe': None,
  'custom_metric': None}]

In [29]:
n_of_sets = len(last_workout['exercises'][0]['sets'])

In [30]:
n_of_sets

6

In [50]:
class ExerciseData(TypedDict):
    exercise: str
    sets: List[Tuple[int, float]]
    volume: int

In [54]:
def calculate_exercise_volume(exercise_data: List[dict]) -> int:
    total_volume = 0
    for s in exercise_data:
        total_volume += s['weight_kg'] * s['reps'] if s['weight_kg'] and s['reps'] else 0
    
    return total_volume

def structure_workout_data(workout_data: dict) -> List[ExerciseData]:
    schema = []
    for exercice in workout_data['exercises']:

        exercise_volume = calculate_exercise_volume(exercice['sets'])

        exercise_data = {
            "exercise": exercice['title'],
            "sets": [(s['reps'], s['weight_kg']) for s in exercice['sets']],
            "volume": exercise_volume
        }
        schema.append(exercise_data)
    return schema

In [55]:
schema = structure_workout_data(last_workout)

In [56]:
schema

[{'exercise': 'Leg Press Horizontal (Machine)',
  'sets': [(14, 55), (15, 64), (17, 82), (14, 91), (12, 100), (12, 100)],
  'volume': 6798},
 {'exercise': 'Leg Extension (Machine)',
  'sets': [(12, 41), (12, 46), (12, 46), (12, 46)],
  'volume': 2148},
 {'exercise': 'Lying Leg Curl (Machine)',
  'sets': [(12, 25), (12, 30), (12, 30), (12, 30)],
  'volume': 1380},
 {'exercise': 'Bicep Curl (Cable)',
  'sets': [(10, 15), (10, 15), (12, 17.5)],
  'volume': 510.0},
 {'exercise': 'Triceps Extension (Cable)',
  'sets': [(10, 15), (10, 15), (12, 17.5)],
  'volume': 510.0},
 {'exercise': 'Dead Hang', 'sets': [(None, None)], 'volume': 0}]

In [ ]:

import math
def get_options(data: ExerciseData, volume_perc: float) -> Dict[str, str]:
    """
    Given one exercise block and a desired % volume increase, return
    pure weight, reps or sets adjustments to hit that new volume.

    Parameters:
    -----------
    data : dict
      {
        'exercise': str,
        'sets': [(reps1, weight1), (reps2, weight2), ...],
        'volume': float    # current total volume
      }
    volume_perc : float
      e.g. 0.10 for +10%

    Returns:
    --------
    {
      'weight': '+X.XXkg',
      'reps'  : '+Y reps',
      'sets'  : '+Z sets'
    }
    """
    current_vol = data['volume']
    target_vol  = current_vol * (1 + volume_perc)
    delta       = target_vol - current_vol

    # pull out only the valid reps & weights
    reps   = [r for r, w in data['sets'] if r is not None]
    weights= [w for r, w in data['sets'] if w is not None]

    total_reps   = sum(reps)
    total_weight = sum(weights)

    # 1) weight increase across all sets: solve x * total_reps = delta
    weight_inc = (delta / total_reps) if total_reps else 0.0

    # 2) reps increase across all sets: solve y * total_weight = delta 
    reps_inc   = (delta / total_weight) if total_weight else 0.0

    # 3) extra sets at average reps & weight: solve z * (avg_r * avg_w) = delta
    avg_r = (total_reps   / len(reps))    if reps    else 0.0
    avg_w = (total_weight / len(weights)) if weights else 0.0
    sets_inc  = (delta / (avg_r * avg_w)) if (avg_r and avg_w) else 0.0

    return {
        'weight': f"+{weight_inc:.2f}kg",
        'reps'  : f"+{math.ceil(reps_inc)} reps",
        'sets'  : f"+{math.ceil(sets_inc)} sets"
    }



In [63]:
get_options(data=schema[0], volume_perc=0.1)

{'weight': '+8.09kg', 'reps': '+2 reps', 'sets': '+1 sets'}

In [68]:
import math
from typing import Dict, Any, List, Tuple

def get_options(data: ExerciseData, volume_perc: float) -> Dict[str, Any]:
    """
    Given one exercise block and a desired % volume increase, return
    weight, reps or sets adjustments to hit that new volume, with guardrails:
    
      1. Round weight-bumps up to 1.25 kg increments
      2. If weight bump > 5 kg → fall back to reps
      3. Cap reps bump at 12 → if > 12 → fall back to sets
      4. When adding sets, show exactly what the new set(s) should look like
    
    Returns a dict:
      {
         'weight': str,       # either "+2.50kg" or "N/A, >5kg bump"
         'reps'  : str,       # either "+3 reps"   or "N/A, >12 reps"
         'sets'  : {
             'count': int,         # how many extra sets
             'plan' : List[Tuple[int, float]]  # each new set as (reps, kg)
         }
      }
    """
    current_vol = data['volume']
    target_vol  = current_vol * (1 + volume_perc)
    delta       = target_vol - current_vol

    # extract valid reps/weights
    reps   = [r for r, w in data['sets'] if r is not None]
    weights= [w for r, w in data['sets'] if w is not None]

    total_reps   = sum(reps)
    total_weight = sum(weights)

    # average reps & weight
    avg_r = total_reps   / len(reps)    if reps    else 0.0
    avg_w = total_weight / len(weights) if weights else 0.0

    # 1) weight-only bump: solve x * total_reps = delta
    raw_w_inc = delta / total_reps if total_reps else 0.0
    # round up to nearest 1.25
    w_inc_rounded = math.ceil(raw_w_inc / 1.25) * 1.25
    if w_inc_rounded <= 5:
        weight_str = f"+{w_inc_rounded:.2f}kg"
    else:
        weight_str = "N/A (>5 kg required)"

    # 2) reps-only bump: solve y * total_weight = delta
    raw_r_inc = delta / total_weight if total_weight else 0.0
    r_inc_ceil = math.ceil(raw_r_inc)
    if r_inc_ceil <= 12 and weight_str.startswith("+"):
        reps_str = f"+{r_inc_ceil} reps"
    elif r_inc_ceil <= 12:
        # even if weight wasn't viable, reps might be
        reps_str = f"+{r_inc_ceil} reps"
    else:
        reps_str = "N/A (>12 reps)"

    # 3) sets-only bump: solve z * (avg_r * avg_w) = delta
    if avg_r > 0 and avg_w > 0:
        raw_s_inc = delta / (avg_r * avg_w)
        s_inc = math.ceil(raw_s_inc)
    else:
        s_inc = 0

    # build plan for each new set
    # round reps to nearest int, weight to 2 decimals
    new_set_template: Tuple[int, float] = (int(round(avg_r)), round(avg_w, 2))
    plan: List[Tuple[int, float]] = [new_set_template for _ in range(s_inc)]

    return {
        'weight': weight_str,
        'reps'  : reps_str,
        'sets'  : {
            'count': s_inc,
            'plan' : plan
        }
    }


In [73]:
get_options(data=schema[0], volume_perc=0.05)

{'weight': '+5.00kg',
 'reps': '+1 reps',
 'sets': {'count': 1, 'plan': [(14, 82.0)]}}

In [77]:
import math
from typing import Dict, Any, List, Tuple

def get_options(data: Dict[str, Any], volume_perc: float) -> Dict[str, Any]:
    """
    Given one exercise block and a desired % volume increase, compute three “minimal”
    adjustments—weight, reps or sets—such that you hit (but don’t exceed) the target,
    subject to guardrails:
    
      • weight bumps rounded up to 1.25 kg
      • if weight bump > 5kg → skip weight strategy
      • reps bump capped at 12 per set → skip reps strategy if exceeded
      • when adding sets, return exactly what each new set should look like
    
    Returns:
      {
        'weight': {
            'bump': float,             # kg per set
            'sets': List[int]          # which set-indices to bump
        } or None,
        'reps': {
            'bump_by_set': List[int]   # +reps per existing set
        } or None,
        'sets': {
            'count': int,              # # new sets
            'plan': List[(reps, kg)]   # what each new set should be
        } or None
      }
    """
    current_vol = data['volume']
    target_vol  = current_vol * (1 + volume_perc)
    delta       = target_vol - current_vol

    # extract valid sets
    sets: List[Tuple[int, float]] = [
        (r, w) for (r, w) in data['sets']
        if r is not None and w is not None
    ]
    n_sets = len(sets)
    reps   = [r for r, _ in sets]
    weights= [w for _, w in sets]

    total_reps   = sum(reps)
    total_weight = sum(weights)
    avg_r = total_reps / n_sets
    avg_w = total_weight / n_sets

    # --- 1) WEIGHT strategy: ---
    # compute the minimal bump (rounded up to 1.25kg) to every set,
    # then see how many sets we actually need to bump.
    weight_option = None
    if total_reps > 0:
        raw_w_inc = delta / total_reps
        w_inc = math.ceil(raw_w_inc / 1.25) * 1.25
        if w_inc <= 5:
            # sort sets by reps desc, pick k sets until volume gain >= delta
            sorted_idx = sorted(range(n_sets), key=lambda i: reps[i], reverse=True)
            cum = 0.0
            bump_sets: List[int] = []
            for i in sorted_idx:
                cum += reps[i] * w_inc
                bump_sets.append(i+1)
                if cum >= delta:
                    break
            weight_option = {'bump': w_inc, 'sets': bump_sets}

    # --- 2) REPS strategy: ---
    # compute total extra reps needed at avg weight,
    # then spread integer reps across sets (no set >12 new reps).
    reps_option = None
    if avg_w > 0:
        raw_total_reps = delta / avg_w
        total_reps_needed = math.ceil(raw_total_reps)
        # if one-set bump would exceed cap, we skip
        if total_reps_needed <= 12 * n_sets:
            # floor division + remainder distribution
            base = total_reps_needed // n_sets
            rem  = total_reps_needed % n_sets
            bump_by_set = [base + (1 if i < rem else 0) for i in range(n_sets)]
            if max(bump_by_set) <= 12:
                reps_option = {'bump_by_set': bump_by_set}

    # --- 3) SETS strategy: ---
    # how many average‐reps@avg‐weight sets needed?
    sets_option = None
    if avg_r > 0 and avg_w > 0:
        raw_sets = delta / (avg_r * avg_w)
        cnt = math.ceil(raw_sets)
        if cnt > 0:
            plan = [(int(round(avg_r)), round(avg_w,2)) for _ in range(cnt)]
            sets_option = {'count': cnt, 'plan': plan}

    return {
        'weight': weight_option,
        'reps'  : reps_option,
        'sets'  : sets_option
    }


In [84]:
get_options(data=schema[4], volume_perc=0.05)

{'weight': {'bump': 1.25, 'sets': [3, 1]},
 'reps': {'bump_by_set': [1, 1, 0]},
 'sets': {'count': 1, 'plan': [(11, 15.83)]}}

In [89]:
import math
from typing import Dict, Any, List, Tuple

def get_options(data: Dict[str, Any], volume_perc: float) -> Dict[str, Any]:
    """
    Compute three minimal strategies—weight, reps or sets—plus debug info,
    and in the “sets” branch *cap* every set at 12 reps for symmetry.
    """
    # 1) Base numbers
    current_vol = data['volume']
    target_vol  = current_vol * (1 + volume_perc)
    delta       = target_vol - current_vol

    # 2) Original “real” sets
    orig_sets: List[Tuple[int, float]] = [
        (r, w) for (r, w) in data['sets']
        if r is not None and w is not None
    ]
    n = len(orig_sets)
    reps   = [r for r, _ in orig_sets]
    weights= [w for _, w in orig_sets]
    total_reps   = sum(reps)
    total_weight = sum(weights)
    avg_r = total_reps / n
    avg_w = total_weight / n

    # 3) Weight strategy (unchanged)
    weight_opt = None
    if total_reps > 0:
        raw_w_inc = delta / total_reps
        bump_kg = math.ceil(raw_w_inc / 1.25) * 1.25
        if bump_kg <= 5:
            idx_by_rep = sorted(range(n), key=lambda i: reps[i], reverse=True)
            cum = 0
            bump_idx = []
            for i in idx_by_rep:
                cum += reps[i] * bump_kg
                bump_idx.append(i)
                if cum >= delta:
                    break
            new_sets = [
                (r, w + bump_kg if i in bump_idx else w)
                for i, (r, w) in enumerate(orig_sets)
            ]
            weight_opt = {
                'bump_kg': bump_kg,
                'sets_idx': bump_idx,
                'added_volume': cum,
                'new_sets': new_sets
            }

    # 4) Reps strategy (unchanged)
    reps_opt = None
    if avg_w > 0:
        raw_total_reps = delta / avg_w
        need_reps = math.ceil(raw_total_reps)
        if need_reps <= 12 * n:
            base = need_reps // n
            rem  = need_reps % n
            bump_by_set = [base + (1 if i < rem else 0) for i in range(n)]
            new_sets = []
            added = 0
            for (r, w), bump in zip(orig_sets, bump_by_set):
                new_sets.append((r + bump, w))
                added += bump * w
            reps_opt = {
                'bump_by_set': bump_by_set,
                'added_volume': added,
                'new_sets': new_sets
            }

    # 5) Sets strategy WITH 12-rep capping
    sets_opt = None
    if avg_r > 0 and avg_w > 0:
        raw_sets = delta / (avg_r * avg_w)
        cnt = math.ceil(raw_sets)
        if cnt > 0:
            # --- raw plan ---
            plan = [(int(round(avg_r)), round(avg_w, 2)) for _ in range(cnt)]
            added_raw = cnt * avg_r * avg_w
            new_full_raw = orig_sets + plan

            # --- cap every set over 12 reps down to 12 ---
            cap_info = []
            new_full_capped = []
            for idx, (r, w) in enumerate(new_full_raw):
                if r > 12:
                    cap_info.append({'idx': idx, 'old_r': r, 'new_r': 12})
                    new_full_capped.append((12, w))
                else:
                    new_full_capped.append((r, w))

            # recompute final added volume
            total_after = sum(r * w for r, w in new_full_capped)
            added_capped = total_after - current_vol

            sets_opt = {
                'count': cnt,
                'plan': plan,
                'raw_added_volume': added_raw,
                'new_full_raw': new_full_raw,
                'capped_info': cap_info,
                'new_full_capped': new_full_capped,
                'capped_added_volume': added_capped
            }

    return {
        'delta': delta,
        'target_volume': target_vol,
        'weight': weight_opt,
        'reps': reps_opt,
        'sets': sets_opt
    }


def pretty_print_options(name: str, opts: Dict[str, Any]) -> None:
    print(f"-- {name} --")
    print(f"Target +{opts['delta']:.1f} → {opts['target_volume']:.1f}\n")

    # Weight
    if opts['weight']:
        w = opts['weight']
        print("Option 1 (increase Weight):")
        print(f"  +{w['bump_kg']} kg on sets {w['sets_idx']}  → +{w['added_volume']:.1f}")
        print(f"  New sets: {w['new_sets']}\n")
    else:
        print("Option 1 (increase Weight): N/A\n")

    # Reps
    if opts['reps']:
        r = opts['reps']
        print("Option 2 (increase Reps):")
        print(f"  bump reps per set: {r['bump_by_set']}  → +{r['added_volume']:.1f}")
        print(f"  New sets: {r['new_sets']}\n")
    else:
        print("Option 2 (increase Reps): N/A\n")

    # Sets
    if opts['sets']:
        s = opts['sets']
        print("Option 3 (add Sets):")
        print(f"  add {s['count']} set(s) {s['plan']}  → raw +{s['raw_added_volume']:.1f}")
        if s['capped_info']:
            print("    capped reps to 12 on:")
            for ci in s['capped_info']:
                print(f"      set #{ci['idx']}: {ci['old_r']} → {ci['new_r']}")
            print(f"    final added volume: +{s['capped_added_volume']:.1f}")
        print(f"  New full session: {s['new_full_capped']}\n")
    else:
        print("Option 3 (add Sets): N/A\n")


In [92]:
ex_session = schema[0]

opts = get_options(ex_session, 0.05)
pretty_print_options(ex_session['exercise'], opts)

-- Leg Press Horizontal (Machine) --
Target +339.9 → 7137.9

Option 1 (increase Weight):
  +5.0 kg on sets [2, 1, 0, 3, 4]  → +360.0
  New sets: [(14, 60.0), (15, 69.0), (17, 87.0), (14, 96.0), (12, 105.0), (12, 100)]

Option 2 (increase Reps):
  bump reps per set: [1, 1, 1, 1, 1, 0]  → +392.0
  New sets: [(15, 55), (16, 64), (18, 82), (15, 91), (13, 100), (12, 100)]

Option 3 (add Sets):
  add 1 set(s) [(14, 82.0)]  → raw +1148.0
    capped reps to 12 on:
      set #0: 14 → 12
      set #1: 15 → 12
      set #2: 17 → 12
      set #3: 14 → 12
      set #6: 14 → 12
    final added volume: +90.0
  New full session: [(12, 55), (12, 64), (12, 82), (12, 91), (12, 100), (12, 100), (12, 82.0)]



In [93]:
ex_session = schema[1]

opts = get_options(ex_session, 0.05)
pretty_print_options(ex_session['exercise'], opts)

-- Leg Extension (Machine) --
Target +107.4 → 2255.4

Option 1 (increase Weight):
  +2.5 kg on sets [0, 1, 2, 3]  → +120.0
  New sets: [(12, 43.5), (12, 48.5), (12, 48.5), (12, 48.5)]

Option 2 (increase Reps):
  bump reps per set: [1, 1, 1, 0]  → +133.0
  New sets: [(13, 41), (13, 46), (13, 46), (12, 46)]

Option 3 (add Sets):
  add 1 set(s) [(12, 44.75)]  → raw +537.0
  New full session: [(12, 41), (12, 46), (12, 46), (12, 46), (12, 44.75)]



In [94]:
import math
from typing import Dict, Any, List, Tuple

def get_options(data: Dict[str, Any], volume_perc: float) -> Dict[str, Any]:
    """
    Compute three minimal‐overshoot strategies—weight, reps or sets—
    including debug info so you can see exact added volumes & new session.

    Returns {
      'delta': float,
      'target_volume': float,
      'weight':   {…} or None,
      'reps':     {…} or None,
      'sets':     {…} or None
    }
    """
    # 1) Base numbers
    current_vol  = data['volume']
    target_vol   = current_vol * (1 + volume_perc)
    delta        = target_vol - current_vol

    # 2) Clean sets
    orig_sets: List[Tuple[int,float]] = [
        (r,w) for (r,w) in data['sets'] if r is not None and w is not None
    ]
    n = len(orig_sets)
    reps    = [r for r,_ in orig_sets]
    weights = [w for _,w in orig_sets]
    total_reps   = sum(reps)
    total_weight = sum(weights)
    avg_r = total_reps / n
    avg_w = total_weight / n

    # --- 3) WEIGHT strategy (unchanged) ---
    weight_opt = None
    if total_reps > 0:
        raw_w_inc = delta / total_reps
        bump_kg   = math.ceil(raw_w_inc / 1.25) * 1.25
        if bump_kg <= 5:
            # choose highest‐rep sets first
            idx_by_rep = sorted(range(n), key=lambda i: reps[i], reverse=True)
            cum = 0.0
            bump_idx = []
            for i in idx_by_rep:
                gain = reps[i] * bump_kg
                cum += gain
                bump_idx.append(i)
                if cum >= delta:
                    break

            new_sets = [
                (r, w + bump_kg if i in bump_idx else w)
                for i, (r,w) in enumerate(orig_sets)
            ]
            weight_opt = {
                'bump_kg': bump_kg,
                'sets_idx': bump_idx,
                'added_volume': cum,
                'new_sets': new_sets
            }

    # --- 4) REPS strategy (greedy by weight) ---
    reps_opt = None
    if total_weight > 0:
        # We'll allocate reps one‐by‐one to the heaviest sets first,
        # never giving any set >12 extra reps.
        bump_by_set = [0]*n
        cum = 0.0
        # we track how many extra reps each set can still take:
        remaining_capacity = [12]*n

        while cum < delta:
            # pick the set with highest w_i that still has capacity
            best = max(
                [(w,i) for i,w in enumerate(weights) if remaining_capacity[i]>0],
                default=(0,None)
            )
            w_i, i = best
            if i is None:
                # no more capacity
                break
            # allocate one rep here
            bump_by_set[i] += 1
            remaining_capacity[i] -= 1
            cum += w_i

        if cum >= delta:
            new_sets = [
                (r + bump_by_set[i], w)
                for i,(r,w) in enumerate(orig_sets)
            ]
            reps_opt = {
                'bump_by_set': bump_by_set,
                'added_volume': cum,
                'new_sets': new_sets
            }

    # --- 5) SETS strategy (only new sets, capped to 12 reps) ---
    sets_opt = None
    if avg_w > 0:
        # decide how many FULL new sets at cap reps
        cap_r     = min(int(round(avg_r)), 12)
        unit_vol  = cap_r * avg_w
        if unit_vol > 0:
            full_sets = int(delta // unit_vol)
            rem_vol   = delta - (full_sets*unit_vol)

            plan: List[Tuple[int,float]] = [(cap_r, round(avg_w,2))]*full_sets

            # if we still need more volume, add one partial set
            if rem_vol > 0:
                # reps needed in partial = ceil( rem_vol / avg_w )
                rp = math.ceil(rem_vol / avg_w)
                rp = min(rp, cap_r)
                plan.append((rp, round(avg_w,2)))

            added = sum(r*w for r,w in plan)
            new_full = orig_sets + plan

            sets_opt = {
                'full_sets': full_sets,
                'partial_set': plan[full_sets:] or None,
                'plan': plan,
                'added_volume': added,
                'new_full_session': new_full
            }

    return {
        'delta': delta,
        'target_volume': target_vol,
        'weight': weight_opt,
        'reps': reps_opt,
        'sets': sets_opt
    }


def pretty_print_options(name: str, opts: Dict[str, Any]) -> None:
    print(f"-- {name} --")
    print(f"Target +{opts['delta']:.1f} → {opts['target_volume']:.1f}\n")

    # Weight
    if opts['weight']:
        w = opts['weight']
        sets_str = ", ".join(str(i) for i in w['sets_idx'])
        print("Option 1 (increase Weight):")
        print(f"  +{w['bump_kg']:.2f}kg on sets [{sets_str}] → +{w['added_volume']:.1f}")
        print("  New sets:", w['new_sets'], "\n")
    else:
        print("Option 1 (increase Weight):  N/A\n")

    # Reps
    if opts['reps']:
        r = opts['reps']
        print("Option 2 (increase Reps):")
        print(f"  bump reps per set: {r['bump_by_set']} → +{r['added_volume']:.1f}")
        print("  New sets:", r['new_sets'], "\n")
    else:
        print("Option 2 (increase Reps):  N/A\n")

    # Sets
    if opts['sets']:
        s = opts['sets']
        print("Option 3 (add Sets):")
        print(f"  full {s['full_sets']}×({s['plan'][0][0]}reps×{s['plan'][0][1]}kg)", end="")
        if s['partial_set']:
            p = s['partial_set'][0]
            print(f" + 1×({p[0]}reps×{p[1]}kg)", end="")
        print(f" → +{s['added_volume']:.1f}")
        print("  New full session:", s['new_full_session'], "\n")
    else:
        print("Option 3 (add Sets):  N/A\n")


In [96]:
ex_session = schema[0]

opts = get_options(ex_session, 0.05)
pretty_print_options(ex_session['exercise'], opts)

-- Leg Press Horizontal (Machine) --
Target +339.9 → 7137.9

Option 1 (increase Weight):
  +5.00kg on sets [2, 1, 0, 3, 4] → +360.0
  New sets: [(14, 60.0), (15, 69.0), (17, 87.0), (14, 96.0), (12, 105.0), (12, 100)] 

Option 2 (increase Reps):
  bump reps per set: [0, 0, 0, 0, 0, 4] → +400.0
  New sets: [(14, 55), (15, 64), (17, 82), (14, 91), (12, 100), (16, 100)] 

Option 3 (add Sets):
  full 0×(5reps×82.0kg) + 1×(5reps×82.0kg) → +410.0
  New full session: [(14, 55), (15, 64), (17, 82), (14, 91), (12, 100), (12, 100), (5, 82.0)] 



In [97]:
import math
from typing import List, Tuple, Dict, Any

def get_optimized_options(
    data: Dict[str, Any],
    volume_perc: float,
    max_pct_weight_bump: float = 0.10,   # never bump >10% of avg_w on phase 1
    rep_floor: int         = 6,
    rep_cap:   int         = 12
) -> Dict[str, Any]:
    """
    A unified 3-phase approach:
      1) Uniform weight bump (capped to max_pct_weight_bump & 1.25kg plates)
      2) Greedy rep add (6 ≤ reps ≤ 12), shifting overflow back into kg bumps
      3) Add full/partial new sets at rep_cap
    
    Returns a dict with each phase’s details & the final new_sets.
    """
    # --- unpack & baselines ---
    orig: List[Tuple[int,float]] = [
        (r,w) for r,w in data['sets'] if r is not None and w is not None
    ]
    n = len(orig)
    reps   = [r for r,_ in orig]
    weights= [w for _,w in orig]
    total_vol = sum(r*w for r,w in orig)
    target_vol= total_vol * (1 + volume_perc)
    delta     = target_vol - total_vol
    avg_w     = sum(weights)/n

    result: Dict[str,Any] = {
        'delta': delta,
        'target_volume': target_vol,
        'phases': []
    }

    # PHASE 1 → weight bump
    raw_w_inc = delta / sum(reps)
    max_w_inc = avg_w * max_pct_weight_bump
    w_inc = min(raw_w_inc, max_w_inc)
    w_inc = math.ceil(w_inc / 1.25) * 1.25
    added_w_vol = sum(r * w_inc for r in reps)
    new_sets = [(r, w + w_inc) for r,w in orig]
    delta2   = delta - added_w_vol

    result['phases'].append({
        'phase': 1,
        'bump_kg': w_inc,
        'added_volume': added_w_vol,
        'new_sets': new_sets,
        'remaining': delta2
    })

    # if we’ve hit it or overshot nicely, stop
    if delta2 <= 0:
        result['final_sets'] = new_sets
        return result

    # PHASE 2 → greedy rep bump
    # allow bump_per_set = rep_cap - original_reps
    cap_per_set = [rep_cap - r for r in reps]
    bump_reps = [0]*n
    cum = 0.0
    # always add reps to the heaviest-set first
    order = sorted(range(n), key=lambda i: new_sets[i][1], reverse=True)
    for i in order:
        while cap_per_set[i] > 0 and cum < delta2:
            bump_reps[i] += 1
            cap_per_set[i] -= 1
            cum += new_sets[i][1]
        if cum >= delta2:
            break

    new_sets2 = [
        (r + bump_reps[i], w) for i,(r,w) in enumerate(new_sets)
    ]
    delta3 = delta2 - cum

    result['phases'].append({
        'phase': 2,
        'bump_reps': bump_reps,
        'added_volume': cum,
        'new_sets': new_sets2,
        'remaining': delta3
    })

    if delta3 <= 0:
        result['final_sets'] = new_sets2
        return result

    # PHASE 3 → new sets at rep_cap
    unit_vol = rep_cap * avg_w
    full_sets = int(delta3 // unit_vol)
    rem_vol   = delta3 - full_sets*unit_vol

    plan: List[Tuple[int,float]] = []
    for _ in range(full_sets):
        plan.append((rep_cap, round(avg_w,2)))

    if rem_vol > 0:
        # partial set: as many reps up to rep_cap
        rep_needed = math.ceil(rem_vol / avg_w)
        rep_needed = min(rep_needed, rep_cap)
        plan.append((rep_needed, round(avg_w,2)))

    added3 = sum(r*w for r,w in plan)
    final = new_sets2 + plan

    result['phases'].append({
        'phase': 3,
        'full_sets': full_sets,
        'partial': plan[full_sets:] or None,
        'added_volume': added3,
        'new_sets': final
    })

    result['final_sets'] = final
    return result


In [98]:
ex_session = schema[0]


opts = get_optimized_options(ex_session, 0.05)
for p in opts['phases']:
    print(f"Phase {p['phase']}: added {p['added_volume']:.1f}, rem {p['remaining']:.1f}")
print("→ final plan:", opts['final_sets'])

Phase 1: added 420.0, rem -80.1
→ final plan: [(14, 60.0), (15, 69.0), (17, 87.0), (14, 96.0), (12, 105.0), (12, 105.0)]


In [114]:
import math
from typing import List, Tuple, Dict, Any, Optional

def optimize_weight_and_reps(
    base_sets: List[Tuple[int,float]],
    delta: float,
    avg_w: float,
    max_pct_wb: float,
    rep_floor: int,
    rep_cap:   int,
    exercise_name: str
) -> Optional[Dict[str,Any]]:
    """
    Try all 1.25kg bumps up to max_pct_wb * avg_w (or only 9kg for Leg Press),
    plus greedy-rep, to hit or slightly overshoot delta.
    """
    n = len(base_sets)
    # pick weight bump candidates
    if exercise_name == 'Leg Press Horizontal (Machine)':
        candidates = [9.0]
    else:
        max_w_inc = math.floor((avg_w*max_pct_wb)/1.25)*1.25
        candidates = [1.25 * i for i in range(1, int(max_w_inc/1.25)+1)]

    best = None
    for w_inc in candidates:
        # apply uniform weight bump
        bumped = [(r, w + w_inc) for r,w in base_sets]
        added_wvol = sum(r * w_inc for r,_ in base_sets)
        rem1 = delta - added_wvol
        if rem1 <= 0:
            return {
                'w_inc': w_inc,
                'bump_reps': [0]*n,
                'total_added': added_wvol,
                'overshoot': added_wvol - delta,
                'final_sets': bumped
            }

        # greedy‐rep on bumped
        bump_r = [0]*n
        cap_extra = [rep_cap - r for r,_ in base_sets]
        # enforce rep_floor
        repfloor_added = 0.0
        for i,(r,w) in enumerate(bumped):
            if r < rep_floor:
                need = min(rep_floor - r, cap_extra[i])
                bump_r[i] = need
                cap_extra[i] -= need
                repfloor_added += need * w
        rem2 = rem1 - repfloor_added

        # one-rep at a time on heaviest
        added_rvol = repfloor_added
        order = sorted(range(n), key=lambda i: bumped[i][1], reverse=True)
        while rem2 > 0:
            for i in order:
                if cap_extra[i] > 0:
                    bump_r[i] += 1
                    cap_extra[i] -= 1
                    added_rvol += bumped[i][1]
                    rem2 -= bumped[i][1]
                    break
            else:
                break

        total_added = added_wvol + added_rvol
        overshoot = total_added - delta

        if any(bump_r) and overshoot >= -1e-6:
            final = [(r + bump_r[i], w + w_inc) for i,(r,w) in enumerate(base_sets)]
            cand = {
                'w_inc': w_inc,
                'bump_reps': bump_r,
                'total_added': total_added,
                'overshoot': overshoot,
                'final_sets': final
            }
            if (best is None or
                cand['overshoot'] < best['overshoot'] - 1e-6 or
               (abs(cand['overshoot']-best['overshoot'])<1e-6 and w_inc < best['w_inc'])):
                best = cand

    return best

def get_optimized_options(
    data: Dict[str,Any],
    volume_perc: float,
    max_pct_weight_bump: float = 0.10,
    rep_floor: int = 6,
    rep_cap:   int = 12
) -> Dict[str,Any]:
    orig = [(r,w) for r,w in data['sets'] if r is not None and w is not None]
    total_vol = sum(r*w for r,w in orig)
    target_vol = total_vol * (1 + volume_perc)
    delta = target_vol - total_vol
    n = len(orig)
    avg_w = sum(w for _,w in orig) / n
    ex_name = data.get('exercise','')

    result = {'delta': delta, 'target_volume': target_vol, 'phases': []}

    # Phase 0: flatten >rep_cap
    lost = 0.0
    base_sets = []
    for r,w in orig:
        if r > rep_cap:
            lost += (r - rep_cap)*w
            base_sets.append((rep_cap,w))
        else:
            base_sets.append((r,w))
    vol0 = sum(r*w for r,w in base_sets)
    rem0 = target_vol - vol0
    result['phases'].append({
        'phase': 0,
        'lost_volume': lost,
        'base_sets': base_sets,
        'remaining': rem0
    })
    if rem0 <= 1e-6:
        result['final_sets'] = base_sets
        return result

    # Phase 1+2: optimized weight+reps
    best = optimize_weight_and_reps(
        base_sets, rem0, avg_w, max_pct_weight_bump,
        rep_floor, rep_cap, ex_name
    )
    if best:
        rem1 = rem0 - best['total_added']
        result['phases'].append({
            'phase': 1,
            'w_inc': best['w_inc'],
            'bump_reps': best['bump_reps'],
            'added_volume': best['total_added'],
            'overshoot': best['overshoot'],
            'final_sets': best['final_sets'],
            'remaining': rem1
        })
        if rem1 <= 1e-6:
            result['final_sets'] = best['final_sets']
            return result
        sets_so_far = best['final_sets']
    else:
        rem1 = rem0
        sets_so_far = base_sets

    # Phase 3: add new sets at rep_cap
    unit = rep_cap * avg_w
    full = int(rem1 // unit)
    rem3 = rem1 - full*unit
    plan = [(rep_cap, round(avg_w,2)) for _ in range(full)]
    if rem3 > 1e-6:
        rp = min(rep_cap, math.ceil(rem3/avg_w))
        plan.append((rp, round(avg_w,2)))
    added3 = sum(r*w for r,w in plan)
    final = sets_so_far + plan

    result['phases'].append({
        'phase': 2,
        'full_new_sets': full,
        'partial_new_set': plan[full:] or None,
        'added_volume': added3,
        'new_sets': final,
        'remaining': rem1 - added3
    })
    result['final_sets'] = final
    return result


In [116]:
ex_session = schema[1]


opts = get_optimized_options(ex_session, 0.05)
for p in opts['phases']:
    print(p)
print("Final session:", opts['final_sets'])

{'phase': 0, 'lost_volume': 0.0, 'base_sets': [(12, 41), (12, 46), (12, 46), (12, 46)], 'remaining': 107.40000000000009}
{'phase': 1, 'w_inc': 2.5, 'bump_reps': [0, 0, 0, 0], 'added_volume': 120.0, 'overshoot': 12.599999999999909, 'final_sets': [(12, 43.5), (12, 48.5), (12, 48.5), (12, 48.5)], 'remaining': -12.599999999999909}
Final session: [(12, 43.5), (12, 48.5), (12, 48.5), (12, 48.5)]
